# 📥 Torrent → Google Drive Downloader (libtorrent 2.x, Colab-ready)

This notebook uses the modern libtorrent 2.x API (declarative settings via
a settings pack or constructor) and includes fallbacks to avoid runtime
errors on Google Colab. It installs a verified wheel (libtorrent==2.0.11)
when needed and uses a fail‑safe session creation pattern.

⚠️ Only download content you are legally allowed to download.

Tips:
- If Drive mount fails, the notebook will save files to `/content/torrents/`.
- This implementation focuses on the core torrent engine (no zipping
  or extra wrappers). It injects public trackers into magnet links by
  default to improve peer discovery in NAT-constrained Colab.

I validated the approach against libtorrent docs and PyPI (libtorrent 2.0.11
wheels exist for CPython 3.12+). ✅

In [ ]:
# Mount Google Drive (optional, robust fallback)
import os

drive_mounted = False
IN_COLAB = False
try:
    from google.colab import drive as _colab_drive
    IN_COLAB = True
except Exception:
    _colab_drive = None
    IN_COLAB = False

try:
    if IN_COLAB:
        if not os.path.exists('/content/drive/MyDrive'):
            _colab_drive.mount('/content/drive', force_remount=True)
        drive_mounted = os.path.exists('/content/drive/MyDrive')
        if drive_mounted:
            print('✅ Google Drive mounted: /content/drive/MyDrive')
except Exception as e:
    print('⚠️ Google Drive mount failed:', e)
    drive_mounted = False

if not drive_mounted:
    os.makedirs('/content/torrents', exist_ok=True)
    print('Using local path: /content/torrents')
else:
    # create a default folder inside MyDrive for convenience
    default_gdrive_path = '/content/drive/MyDrive/Torrent'
    os.makedirs(default_gdrive_path, exist_ok=True)
    print('Using Google Drive path:', default_gdrive_path)


In [ ]:
# Install libtorrent (2.0.11) and requests if missing
import sys
import subprocess
import importlib

def pip_install(pkg):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])
    except subprocess.CalledProcessError as e:
        print(f'pip install failed for {pkg}:', e)
        raise

lt = None
try:
    import libtorrent as lt
    print('✅ libtorrent present:', getattr(lt, 'version', getattr(lt, '__version__', 'unknown')))
except Exception:
    print('⚠️ libtorrent not found. Trying libtorrent==2.0.11 ...')
    try:
        pip_install('libtorrent==2.0.11')
        importlib.invalidate_caches()
        import libtorrent as lt
        print('✅ libtorrent installed:', getattr(lt, 'version', getattr(lt, '__version__', 'unknown')))
    except Exception as e:
        print('Primary install failed, trying alternative wheel name py3-libtorrent...')
        pip_install('py3-libtorrent')
        importlib.invalidate_caches()
        import libtorrent as lt
        print('✅ libtorrent installed via py3-libtorrent:', getattr(lt, 'version', getattr(lt, '__version__', 'unknown')))

try:
    import requests
except Exception:
    print('Installing requests...')
    pip_install('requests')
    importlib.invalidate_caches()
    import requests
    print('✅ requests installed')


In [ ]:
# Core modern, Colab-tuned torrent engine (settings constructor + settings_pack fallback)
import libtorrent as lt
import time
import os
import urllib.parse
import math
import shutil

def _create_session(settings: dict):
    """Try lt.session(settings) first; fallback to settings_pack + apply_settings.
    This handles differences across Python bindings/wheels.
    """
    try:
        ses = lt.session(settings)
        return ses
    except TypeError:
        sp = lt.settings_pack()
        for k, v in settings.items():
            # try to map a string key to the settings_pack enum
            try:
                key = getattr(lt.settings_pack, k)
            except Exception:
                # some bindings expose setting_by_name
                try:
                    key = lt.settings_pack.setting_by_name(k)
                except Exception:
                    continue
            if isinstance(v, bool):
                sp.set_bool(key, v)
            elif isinstance(v, int):
                sp.set_int(key, v)
            else:
                sp.set_str(key, str(v))
        ses = lt.session()
        ses.apply_settings(sp)
        return ses

def _add_trackers_to_magnet(magnet: str, trackers: list) -> str:
    if not magnet.startswith('magnet:'):
        return magnet
    for tr in trackers:
        magnet += '&tr=' + urllib.parse.quote(tr)
    return magnet

def _fmt_eta(remaining_bytes, rate):
    if rate <= 0: return 'ETA: ∞'
    secs = int(remaining_bytes / rate)
    m, s = divmod(secs, 60)
    h, m = divmod(m, 60)
    if h: return f'ETA: {h}h {m}m'
    if m: return f'ETA: {m}m {s}s'
    return f'ETA: {s}s'

def download_colab(magnet_link: str, save_path: str,
                   max_peers: int = 400, peer_connect_timeout: int = 3,
                   add_trackers: bool = True,
                   stall_timeout: int = 900,  # 15 min with near-zero progress
                   auto_zip: bool = False, zip_name=None):
    """Download a magnet to save_path using modern libtorrent 2.x patterns.
    Prints status every 5s. Exits when seeding. Optionally zips output.
    """
    # optional public trackers (helps tracker-less magnets)
    if add_trackers:
        trackers = [
            'udp://tracker.opentrackr.org:1337/announce',
            'udp://tracker.torrent.eu.org:451/announce',
            'udp://exodus.desync.com:6969/announce',
            'udp://tracker.openbittorrent.com:6969/announce'
        ]
        magnet_link = _add_trackers_to_magnet(magnet_link, trackers)

    settings = {
        'user_agent': f'libtorrent/{getattr(lt, "version", "unknown")}',
        'listen_interfaces': '0.0.0.0:6881',
        'enable_dht': True,
        'enable_lsd': True,
        'announce_to_all_trackers': True,
        'announce_to_all_tiers': True,
        'connections_limit': max_peers,
        'max_peerlist_size': max_peers * 2,
        'peer_connect_timeout': peer_connect_timeout,
        'request_timeout': 10,
        'min_reconnect_time': 5,
        'active_downloads': -1,
        'active_seeds': -1,
        'allow_multiple_connections_per_ip': True,
    }
    ses = _create_session(settings)

    params = lt.parse_magnet_uri(magnet_link)
    params.save_path = save_path
    # sequential download flag set from the start
    params.flags |= lt.torrent_flags.sequential_download

    h = ses.add_torrent(params)
    print('🔗', magnet_link)

    # wait for metadata (magnet) with a soft timeout to avoid hanging forever
    meta_wait = 0
    while not h.status().has_metadata:
        print('📡 fetching metadata...')
        time.sleep(1)
        meta_wait += 1
        if meta_wait > 120:  # 2 minutes
            print('⚠️ metadata fetch taking too long; continuing anyway')
            break

    s = h.status()
    print('✅ metadata fetched — starting:' if s.has_metadata else '▶️ starting without metadata:', s.name)

    # progress loop with stall detection and ETA
    last_progress = 0.0
    last_change_ts = time.time()
    while not h.status().is_seeding:
        s = h.status()
        total = s.total_wanted if s.total_wanted > 0 else 0
        done = s.total_wanted_done
        remaining = max(0, total - done)
        eta = _fmt_eta(remaining, s.download_rate) if total else 'ETA: ?'
        print(f"{s.progress*100:5.2f}% | ↓{s.download_rate/1000:6.1f} kB/s | ↑{s.upload_rate/1000:6.1f} kB/s | peers {s.num_peers} | {eta}")
        # stall detection: if progress hasn't moved by >0.1% for stall_timeout seconds
        if s.progress - last_progress > 0.001:
            last_progress = s.progress
            last_change_ts = time.time()
        elif stall_timeout and stall_timeout > 0 and (time.time() - last_change_ts > stall_timeout):
            print('⏱️ Download appears stalled. Stopping session to free resources.')
            h.pause()
            break
        time.sleep(5)

    completed = h.status().is_seeding
    if completed:
        print('🎉 COMPLETE — saved to', save_path)
    else:
        print('Stopped before completion.')

    # optional zip (archive only the torrent's files, not entire save_path)
    if auto_zip:
        s = h.status()
        name = s.name or 'torrent'
        base = (zip_name or name).replace(' ', '_')
        target_path = os.path.join(save_path, name)
        if os.path.isdir(target_path):
            root_dir = target_path
        else:
            root_dir = save_path
        zip_target = os.path.join(save_path, base)
        print(f'🗜️ Creating zip: {zip_target}.zip ...')
        shutil.make_archive(zip_target, 'zip', root_dir)
        print('✅ Zip created:', zip_target + '.zip')
